In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from math import sqrt
from scipy.stats import norm
from scipy.stats import uniform
import cmath #for complex numbers
from scipy.integrate import quad #for numerical integration
from sklearn.preprocessing import MinMaxScaler
import scipy

In [3]:
#input:
#output: Call option price with strike K and Maturity T
def heston_closed(a,b,c,T,K,rho,V0,S0,r=0):
    
    def char_f(w,a,b,c,T,K,rho,V0,S0,r):
        alpha = -0.5*w*(w+complex(0,1))
        beta = a - rho*c*w*complex(0,1)
        gamma = c*c/2
        h = cmath.sqrt(beta*beta-4*alpha*gamma)
        rplus = (beta + h)/c/c
        rminus = (beta - h)/c/c
        g = rminus/rplus
        D =  rminus*(1-cmath.exp(-h*T))/(1-g*cmath.exp(-h*T))
        C = a*(rminus*T-2/c/c*cmath.log((1-g*cmath.exp(-h*T))/(1-g)))
        return cmath.exp(C*b+D*V0+complex(0,1)*w*cmath.log(S0*cmath.exp(r*T)))

    def integrand1(w):
        i = complex(0,1)
        return (cmath.exp(-i*w*cmath.log(K))*char_f(w-i,a,b,c,T,K,rho,V0,S0,r)/i/w/char_f(-i,a,b,c,T,K,rho,V0,S0,r)).real
    
    def integrand2(w):
        i = complex(0,1)
        return (cmath.exp(-i*w*cmath.log(K))*char_f(w,a,b,c,T,K,rho,V0,S0,r)/i/w).real
    
    pi1 = 0.5 + quad(integrand1,0,np.inf)[0]/np.pi
    pi2 = 0.5 + quad(integrand2,0,np.inf)[0]/np.pi
    
    return (S0*pi1 + cmath.exp(-r*T)*K*pi2).real

In [4]:
K = [0.4,0.5,0.6,0.7] #strikes
T = [5,6,7,8,9] #maturities
#initial values
V0 = 0.2
S0 = 1

In [5]:
num_input_parameters = 4
num_output_parameters = len(K)*len(T)
learning_rate = 0.001
num_steps = 100
batch_size = 5
num_neurons = int(num_output_parameters/2)

scaler = MinMaxScaler()

X = tf.placeholder(tf.float32, [None, num_input_parameters])
y = tf.placeholder(tf.float32, [None, num_output_parameters])

In [6]:
#input: batch_size, bounds for a,b,c,rho in that order 
#output: X_batch,y_batch
def next_batch_heston_train(batch_size,bounds):
    X = np.zeros((batch_size,num_input_parameters))
    y = np.zeros((batch_size,num_output_parameters))
    
    X[:,0] = uniform.rvs(size=batch_size)*(bounds[0][1]-bounds[0][0]) + bounds[0][0]#a
    X[:,1] = uniform.rvs(size=batch_size)*(bounds[1][1]-bounds[1][0]) + bounds[1][0] #b
    X[:,2] = uniform.rvs(size=batch_size)*(bounds[2][1]-bounds[2][0]) + bounds[2][0] #c
    X[:,3] = uniform.rvs(size=batch_size)*(bounds[3][1]-bounds[3][0]) + bounds[3][0] #rho

    
    for i in range(batch_size):
        for j in range(len(K)):
            for k in range(len(T)):
                y[i,j*len(T)+k] = heston_closed(X[i,0],X[i,1],X[i,2],T[k],K[j],X[i,3],V0,S0)
    
    #X = scaler.fit_transform(X)
    #y = scaler.fit_transform(y)
    
    return X,y

In [7]:
#bounds for a,b,c,rho,T,K
bounds = np.array([[1,3],[0.1,0.6],[0,0.1],[0,1],[1,2],[0.6,1.3]])

In [8]:
#Layers
hidden1 = fully_connected(X, num_neurons, activation_fn=tf.nn.elu)
hidden2 = fully_connected(hidden1, num_neurons, activation_fn=tf.nn.elu)
hidden3 = fully_connected(hidden2, num_neurons, activation_fn=tf.nn.elu)
hidden4 = fully_connected(hidden3, num_neurons, activation_fn=tf.nn.elu)

outputs = fully_connected(hidden4, num_output_parameters, activation_fn=None)

Instructions for updating:
Please use `layer.__call__` method instead.


In [9]:
#Loss Function
loss = tf.reduce_mean(tf.sqrt(tf.square(outputs - y)))  # MSE

In [10]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [11]:
init = tf.global_variables_initializer()

In [12]:
saver = tf.train.Saver()

In [12]:
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(num_steps):
        
        X_batch,Y_batch = next_batch_heston_train(batch_size,bounds)
        sess.run(train,feed_dict={X: X_batch, y: Y_batch})
        
        if iteration % 10 == 0:
            
            rmse = loss.eval(feed_dict={X: X_batch, y: Y_batch})
            print(iteration, "\tRMSE:", rmse)
    
    saver.save(sess, "./models/heston_closed_nn_2")

0 	RMSE: 0.92215383
10 	RMSE: 0.1873601
20 	RMSE: 0.10427293
30 	RMSE: 0.11149052
40 	RMSE: 0.059611436
50 	RMSE: 0.105607964
60 	RMSE: 0.07302338
70 	RMSE: 0.06603402
80 	RMSE: 0.058842324
90 	RMSE: 0.04150214


In [13]:
def NNprediction(theta):
    x = np.zeros((1,len(theta)))
    x[0,:] = theta
    return sess.run(outputs,feed_dict={X: x})
def NNgradientpred(x):
    x = np.asarray(x)
    grad = np.zeros((num_output_parameters,num_input_parameters))
    
    delta = 0.000001
    for i in range(num_input_parameters):
        h = np.zeros(x.shape)
        h[0,i] = delta
        
        #two point gradient
        #grad[i] = (sess.run(outputs,feed_dict={X: x+h}) - sess.run(outputs,feed_dict={X: x-h}))/2/delta

        #four point gradient
        grad[:,i] = (-sess.run(outputs,feed_dict={X: x+2*h})+8*sess.run(outputs,feed_dict={X: x+h})-8*sess.run(outputs,feed_dict={X: x-h}) +sess.run(outputs,feed_dict={X: x-2*h}))/12/delta

    return grad.T

In [14]:
theta_true = [2.1,3.2,0.8,0.4]


#input: theta in order a,b,c,rho
def CostFuncLS(theta):

    if theta[3]<=0:
        theta[3] = 0.01
    if theta[3]>=1:
        theta[3] = 0.99
    if theta[2]<=0:
        theta[2] = 0.1
    if 2*theta[0]*theta[1]<=theta[2]*theta[2]:
        theta[0] += theta[2]
        theta[1] += theta[2]
   
    heston = np.zeros(num_output_parameters)
    for i in range(len(T)):
        for j in range(len(K)):
            heston[i*len(K)+j] = heston_closed(theta_true[0],theta_true[1],theta_true[2],T[i],K[j],theta_true[3],V0,S0)

    return (NNprediction(theta)-heston)[0]


def JacobianLS(theta):

    if theta[3]<=0:
        theta[3] = 0.01
    if theta[3]>=1:
        theta[3] = 0.99
    if theta[2]<=0:
        theta[2] = 0.1
    if 2*theta[0]*theta[1]<=theta[2]*theta[2]:
        theta[0] += theta[2]
        theta[1] += theta[2]
    x = np.zeros((1,len(theta)))
    x[0,:] = theta
    return NNgradientpred(x).T

In [16]:
with tf.Session() as sess:                          
    saver.restore(sess, "./models/heston_closed_nn_2")      
    
    init = [1.5,0.5,0.05,0.6]
    bnds = ([bounds[0,0],bounds[1,0],bounds[2,0],bounds[3,0]],[bounds[0,1],bounds[1,1],bounds[2,1],bounds[3,1]])


    I=scipy.optimize.least_squares(CostFuncLS,init,JacobianLS,bounds=bnds,gtol=1E-10,xtol=1E-12,verbose=1)
        

INFO:tensorflow:Restoring parameters from ./models/heston_closed_nn_2
`ftol` termination condition is satisfied.
Function evaluations 14, initial cost 1.2765e+00, final cost 1.2709e+00, first-order optimality 1.24e-02.


In [17]:
theta_nn = I.x

In [20]:
with tf.Session() as sess:                          
    saver.restore(sess, "./models/heston_closed_nn_2")      
    
    RMSE = np.sqrt(np.sum(np.power(CostFuncLS(theta_nn),2),axis=0))

print("RMSE: ",RMSE)

INFO:tensorflow:Restoring parameters from ./models/heston_closed_nn_2
RMSE:  1.5943240866539232
